In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

with open('/Users/hibye/.secret/yelp_api.json') as f:
    login = json.load(f)
    login.keys()

In [2]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [3]:
# Quick Test Query
results = yelp_api.search_query(location='Baltimore, MD',
                                       term='Crab Cake')
print(type(results))
results.keys()


<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [4]:

# set our API call parameters 
LOCATION = 'Alqonquin,IL'
TERM = 'Pizza'


In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_pizza_IL.json"
JSON_FILE


'Data/results_in_progress_pizza_IL.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_pizza_IL.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

377

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



19

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/19 [00:00<?, ?it/s]

In [14]:
os.path.isfile(JSON_FILE)

True

In [15]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
## If it DOES exist:
if file_exists == True:
        
    ## Check if user wants to delete if exists
    if delete_if_exists==True:
            
        print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
        ## delete file and confirm it no longer exits.
        os.remove(JSON_FILE)
## Recursive call to function after old file deleted
        create_json_file(JSON_FILE,delete_if_exists=False)
    else:
        print(f"[i] {JSON_FILE} already exists.")            
            
            
## If it does NOT exist:
else:
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  


[i] Data/results_in_progress_pizza_IL.json not found. Saving empty list to new file.


In [18]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)


  0%|          | 0/19 [00:00<?, ?it/s]

In [19]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,ULg5fU6JTOCx4va9eUpjTQ,pizza-pushers-algonquin,Pizza Pushers,https://s3-media1.fl.yelpcdn.com/bphoto/Cs5k7Y...,False,https://www.yelp.com/biz/pizza-pushers-algonqu...,21,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 42.16317, 'longitude': -88.29628}",[],"{'address1': '644 S Main St', 'address2': '', ...",,,1826.303215,NaN
1,4pCvjmcFZMyPUIBqWsls2A,sals-pizza-company-algonquin,Sal's Pizza Company,https://s3-media2.fl.yelpcdn.com/bphoto/X5SBNj...,False,https://www.yelp.com/biz/sals-pizza-company-al...,208,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 42.176214, 'longitude': -88.31533}",[delivery],"{'address1': '5 Hanson Rd', 'address2': '', 'a...",+18476587272,(847) 658-7272,1451.881180,$$
2,2Hzt_Z8SXBsDiOu2vWP0Kg,uncle-jerry-s-pizza-company-cary,Uncle Jerry’s Pizza Company,https://s3-media2.fl.yelpcdn.com/bphoto/8Jlb3l...,False,https://www.yelp.com/biz/uncle-jerry-s-pizza-c...,106,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 42.20837, 'longitude': -88.24259}",[],"{'address1': '133 W Main St', 'address2': '', ...",+12248888663,(224) 888-8663,8016.547087,$$
3,CftHLAdJj63kAVxYQDZehQ,dinos-pizza-and-pasta-lake-in-the-hills,Dino's Pizza & Pasta,https://s3-media3.fl.yelpcdn.com/bphoto/bpeElv...,False,https://www.yelp.com/biz/dinos-pizza-and-pasta...,67,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 42.190623, 'longitude': -88.331221}","[pickup, delivery]","{'address1': '6 Miller Rd', 'address2': '', 'a...",+18476583300,(847) 658-3300,3210.464471,$
4,sYwBAWlDCRe582S4ZfgKQg,woodfire-dundee-west-dundee-2,Woodfire Dundee,https://s3-media2.fl.yelpcdn.com/bphoto/UQrsVC...,False,https://www.yelp.com/biz/woodfire-dundee-west-...,130,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 42.0979661656296, 'longitude': -8...",[delivery],"{'address1': '127 W Main St', 'address2': '', ...",+18478440886,(847) 844-0886,7987.399488,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
372,DTc25oxDLvxJeV3wWWyFZQ,chilis-streamwood-3,Chili's,https://s3-media2.fl.yelpcdn.com/bphoto/0bqITS...,False,https://www.yelp.com/biz/chilis-streamwood-3?a...,82,"[{'alias': 'tex-mex', 'title': 'Tex-Mex'}, {'a...",3.0,"{'latitude': 42.015895470264, 'longitude': -88...","[pickup, delivery]","{'address1': '1041 Sutton Rd', 'address2': Non...",+16304832352,(630) 483-2352,18984.729695,$$
373,BAdDevvE7YYAdMe0BwSXtA,arbys-streamwood,Arby's,https://s3-media4.fl.yelpcdn.com/bphoto/TfsNxG...,False,https://www.yelp.com/biz/arbys-streamwood?adju...,21,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.0,"{'latitude': 42.0214042663574, 'longitude': -8...","[pickup, delivery]","{'address1': '520 S Sutton Rd', 'address2': No...",+16304979109,(630) 497-9109,18413.883793,$
374,PmEmQoPGkQKvd7B6JdCmpQ,taco-bell-elgin-3,Taco Bell,https://s3-media2.fl.yelpcdn.com/bphoto/y8XHZD...,False,https://www.yelp.com/biz/taco-bell-elgin-3?adj...,25,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",2.0,"{'latitude': 42.023532, 'longitude': -88.311785}","[pickup, delivery]","{'address1': '304 South McLean Blvd', 'address...",+18476080842,(847) 608-0842,15557.446433,$
375,es-46dK1zT2kYV6V-BfzxQ,apple-villa-cafe-hoffman-estates,Apple Villa Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/zaI92C...,False,https://www.yelp.com/biz/apple-villa-cafe-hoff...,230,"[{'alias': 'breakfast_brunch', 'title': 'Break...",3.5,"{'latitude': 42.079257, 'longitude': -88.140138}","[pickup, delivery]","{'address1': '3101 N Barrington Rd', 'address2...",+18477652000,(847) 765-2000,17424.771196,$$
376,O5ZltApu7Xq7umLZ4pXCTg,golden-corral-buffet-and-grill-elgin,Golden Corral Buffet & Grill,https://s3-media4.fl.yelpcdn.com/bphoto/ICq_0O...,False,https://www.yelp.com/biz/golden-corral-buffet-...,91,"[{'alias': 'buffets', 'title': 'Buffets'}, {'a...",2.0,"{'latitude': 42.0193983926583, 'longitude': -8...","[pickup, delivery]","{'address1': '601 S Randall Rd', 'address2': '...",+18475317277,(847) 531-7277,16073.849143,$$


In [20]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [23]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_IL_pizza.csv.gz', compression='gzip',index=False)